In [1]:
# import cell

import xarray as xr # to work with multi-dimensional arrays
import numpy as np # to work with multi-dimensional arrays
import pandas as pd
import pickle
import xesmf as xe # to regrid data
import glob # to find file pathways
import netCDF4 # use to save xarrays as nc files
import time

import matplotlib
import matplotlib.pyplot as plt # for plotting
import matplotlib.colors as colors # for custom colourbars
import matplotlib.gridspec as gridspec # for more custom subplot positioning
import matplotlib.ticker as ticker # for custom tick label formatting
import cartopy # use for geographic map projections
import cartopy.crs as ccrs # use for geographic map projections
import regionmask # to work with IPCC sixth assessment regions

### Absolute Heat Exposure: All Models 1x1 Resolution

 as population data constant for each cell for a given model and scenario, can simply scale mean and standard
 deviation of tropical night data to get exposure data, to get the mean and std for exposure

In [ ]:
# load in HYDE 3.2 and NCAR-CIDR populaton 1x1 resolution data
hyde_pw = '/home/ucfagtj/DATA/Dissertation/Data/Population/processed/hyde3_2_pop_variables_1x1_res.nc'
ncar_pw = '/home/ucfagtj/DATA/Dissertation/Data/Population/processed/ncar_pop_variables_1x1_res_with_anom.nc'
hyde_ds, ncar_ds = xr.open_dataset(hyde_pw), xr.open_dataset(ncar_pw)

# combine both population Dataset objects to form single population Dataset
pop_ds = hyde_ds.merge(ncar_ds)
hyde_ds.close(), ncar_ds.close()

# multiply each model's tropical night data by associated population projection to get exposure
for scenario in ['pre_ind','current', 'ssp126', 'ssp245', 'ssp370', 'ssp585']:
    
    # define the pathways for each model's tropical nights 1x1 degree resolution data for given scenario
    f_pws = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/trop_nights/{scenario}/*/*/*absolute*1x1*')
    
    # define the associated population data for given scenario; need to account for differing naming style
    if scenario in ['pre_ind']:
        tot_pc = pop_ds['preind_mean_popc']
    elif scenario in ['current']:
        tot_pc = pop_ds[f'{scenario}_mean_popc']
    else:
        tot_pc = pop_ds[f'{scenario[0: 4]}_mean_totalc']
        
    # load in single model tropical nights data
    for pw in f_pws:
        x = xr.open_dataset(pw, use_cftime = True)
        data = x
        x.close()
        
        # create an empty Dataset object to populate exposure data in
        exp_ds = xr.Dataset()
        
        # compute exposure variables; simply scale by scenario's population projection
        mean_exp = data.mean_ann_tn * tot_pc 
        std_exp = data.std_ann_tn * tot_pc
        
        # populate exposure Dataset
        exp_ds = exp_ds.assign({'mean_exposure': mean_exp})
        exp_ds = exp_ds.assign({'std_exposure': std_exp})
        
        # add attribute information to exposure Dataset
        exp_ds.attrs['long_title'] = 'spatial annual heat exposure: annual tropical nights * population [person/days]'
        exp_ds.attrs['data_type'], exp_ds.attrs['model'] = data.data_type, data.model
        exp_ds.attrs['modelling_group'], exp_ds.attrs['realisation'] = data.modelling_group, data.realisation
        exp_ds.attrs['experiment'], exp_ds.attrs['years_sampled'] = data.experiment, data.years_sampled
        exp_ds.attrs['period'], exp_ds.attrs['resolution'] = data.period, data.resolution
        exp_ds.attrs['created_on'] = time.ctime()
        
        # save exposure Dataset object
        save_pw = pw.rsplit('/', 5)[0] + f'/heat_exposure/{data.experiment}/{data.modelling_group}/{data.model}' + \
                  f'/{data.experiment}_{data.model}_absolute_exposure_1x1_res.nc'
        #exp_ds.to_netcdf(save_pw, mode = 'w')
        print(save_pw)
    
        # close Dataset and DataArray objects
        data.close(), mean_exp.close(), std_exp.close(), exp_ds.close()
        
    # close scenario population DataArray
    tot_pc.close()

# close population Dataset
pop_ds.close()

In [26]:
## rough workingggggg

pw = '/home/ucfagtj/DATA/Dissertation/Data/trop_nights/ssp585/AWI/AWI-CM-1-1-MR/' + \
     'ssp585_AWI-CM-1-1-MR_absolute_ann_tn_2071_2100.nc'
x = xr.open_dataset(pw)
print(x.mean_ann_tn.sum())
x.close()

<xarray.DataArray 'mean_ann_tn' ()>
array(9789234., dtype=float32)


### Heat Exposure Anomaly (Pre-Ind): All Models 1x1 Resolution

As both population datasets are of 1x1 degree spatial resolution, 'raw' tropical nights data must be regridded to same resolution before being converted to heat exposure data. To compute the heat exposure pre-industrial anomaly, negate the mean pre-industrial exposure from each year's heat exposure sum, and then take the time average of this anomaly.

Metacode:

1. Load in HYDE 3.2 and NCAR-CIDR population projections of 1x1 degree resolution.
2. Load in the pre-industrial mean heat exposure.
3. Regrid the 'raw' tropical nights data to a 1x1 degree resolution. ('raw' data is that containing the annual number of tropical nights for every year.)
4. Multiply the regridded tropical nights data by the corresponding mean population count to get heat exposure.
5. Negate the pre-industrial mean heat exposure from each year of the newly computed heat exposure to get the heat exposure anomaly for each year.
6. Compute the time average of this anomaly.
7. Add attribute information.
8. Save resulting dataset.

In [ ]:
# load in HYDE 3.2 and NCAR-CIDR populaton 1x1 resolution data
hyde_pw = '/home/ucfagtj/DATA/Dissertation/Data/Population/processed/hyde3_2_pop_variables_1x1_res.nc'
ncar_pw = '/home/ucfagtj/DATA/Dissertation/Data/Population/processed/ncar_pop_variables_1x1_res_with_anom.nc'
hyde_ds, ncar_ds = xr.open_dataset(hyde_pw), xr.open_dataset(ncar_pw)

# combine both population Dataset objects to form single population Dataset
pop_ds = hyde_ds.merge(ncar_ds)
hyde_ds.close(), ncar_ds.close()

# compute exposure anomaly relative the pre-industrial for a given scenario and model 
for scenario in ['current', 'ssp126', 'ssp245', 'ssp370', 'ssp585']:
    
    # define the population data for given scenario; need to account for differing naming style
    if scenario in ['pre_ind']:
        tot_pc = pop_ds['preind_mean_popc']
    elif scenario in ['current']:
        tot_pc = pop_ds[f'{scenario}_mean_popc']
    else:
        tot_pc = pop_ds[f'{scenario[0: 4]}_mean_totalc']
    
    # define pathways for 'raw' tropical night data for given scenario
    f_pws = glob.glob(f'/home/ucfagtj/DATA/Dissertation/Data/trop_nights/working_files/{scenario}/*')
        
    for pw in f_pws:
         
        # load in the 'raw' tropical night data for given model and scenario
        x = xr.open_dataset(pw, use_cftime = True)
        data = x.tasmin
        data = data.where(data < 365., 365) # restrict upper limit to 365 days; ignore extra day from leap year
        x.close()
        
        # load in the pre-industrial mean exposure for given model and scenario in 1x1 resolution
        preind_pw = glob.glob(pw.rsplit('/', 4)[0] + '/heat_exposure/pre_ind' + \
                               f'/{data.modelling_group}/{data.model}/*absolute*1x1*')[0]
        x = xr.open_dataset(preind_pw, use_cftime = True)
        preind_exp = x.mean_exposure
        x.close()    
        
        
        ######### regridding of 'raw' data to 1x1 resolution ########
        
        
        # define the desired output grid/resolution; 1.0 x 1.0 degrees
        grid_out = xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90.5, 1.)),
                               'lon': (['lon'], np.arange(-179.5, 180.5, 1.))})
        
        # define the input grid
        grid_in = data
        
        # compute the "regridder" file which will define the weighting to apply
        regridder = xe.Regridder(grid_in, grid_out,
                                 method = 'bilinear', # interpolation method
                                 periodic = True) # required for global girds; prevents blank data on meridian
    
        # apply the weighting matrix to transform the data to new resolution
        rg_data = regridder(grid_in)
        grid_in.close(), grid_out.close()
        
        # clear regridder file from being saved
        regridder.clean_weight_file()
        
        
        ######## computation of exposure anomaly ########
        
        
        # compute the spatial exposure for every year within data; multiply by population
        exp = rg_data * tot_pc
        rg_data.close()

        # negate the pre-industrial baseline from each year's annual tropical night
        exp = exp - preind_exp
        preind_exp.close()

        # compute the spatial mean exposure anomaly
        mean_exp_anom = exp.mean(dim = 'year', skipna = True, keep_attrs = True)
        
        # compute the standard deviation of the spatial mean exposure anomaly
        std_exp_anom = exp.std(dim = 'year', skipna = True, keep_attrs = True)
        exp.close()
        
        # create an empty Dataset object to populate exposure data in
        anom_exp_ds = xr.Dataset()
        anom_exp_ds = anom_exp_ds.assign({'mean_exposure_anom': mean_exp_anom})
        anom_exp_ds = anom_exp_ds.assign({'std_exposure_anom': std_exp_anom})
        
        # add attribute information to exposure anomaly Dataset object
        anom_exp_ds.attrs['long_title'] = 'spatial annual heat exposure anomaly relative' + \
                                          ' to the pre-industrial[person/days]'
        anom_exp_ds.attrs['data_type'], anom_exp_ds.attrs['model'] = 'anomaly', data.model
        anom_exp_ds.attrs['modelling_group'], anom_exp_ds.attrs['realisation'] = data.modelling_group, data.realisation
        anom_exp_ds.attrs['experiment'], anom_exp_ds.attrs['years_sampled'] = data.experiment, data.years_aggregated
        anom_exp_ds.attrs['period'], anom_exp_ds.attrs['resolution'] = data.period, 'lon-lat: 1x1 degrees'
        anom_exp_ds.attrs['created_on'], anom_exp_ds.attrs['baseline'] = time.ctime(), '1851-1900'
        
        # save exposure Dataset object
        save_pw = pw.rsplit('/', 4)[0] + f'/heat_exposure/{data.experiment}/{data.modelling_group}' + \
                  f'/{data.model}/{data.experiment}_{data.model}_exposure_anomaly_1x1_res.nc'
        anom_exp_ds.to_netcdf(save_pw, mode = 'w')
        print(save_pw)
        
        # close open Dataset objects
        data.close(), mean_exp_anom.close(), std_exp_anom.close(), anom_exp_ds.close()
        
    # close scenario population DataArray
    tot_pc.close()

# close population Dataset
pop_ds.close()

### Absolute Heat Exposure and Heat Exposure Anomaly Dataset: Multi-Model Ensemble

In [17]:
anom_cur = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/current/*/*/*anomaly*_1x1*')
anom_126 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/ssp126/*/*/*anomaly*_1x1*')
anom_245 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/ssp245/*/*/*anomaly*_1x1*')
anom_370 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/ssp370/*/*/*anomaly*_1x1*')
anom_585 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/ssp585/*/*/*anomaly*_1x1*')
abso_pre = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/pre_ind/*/*/*abso*_1x1*')
abso_cur = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/current/*/*/*abso*_1x1*')
abso_126 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/ssp126/*/*/*abso*_1x1*')
abso_245 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/ssp245/*/*/*abso*_1x1*')
abso_370 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/ssp370/*/*/*abso*_1x1*')
abso_585 = glob.glob('/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/ssp585/*/*/*abso*_1x1*')

f_pws_list = [anom_cur, anom_126, anom_245, anom_370, anom_585,
              abso_pre, abso_cur, abso_126, abso_245, abso_370, abso_585]

In [19]:
# create Dataset object to store the spatial ensemble annual heat exposure variables
ens_ds = xr.Dataset()

for pws_group in f_pws_list:
    
    # load in a single model output; remaining model outputs will be concatenated to this
    x0 = xr.open_dataset(pws_group[0], use_cftime = True)
    models = [x0.model] # note model used
    
    # concatenate remaining model ouputs along a new dimension
    for pw in pws_group[1:]:
        x = xr.open_dataset(pw, use_cftime = True)
        x0 = xr.concat([x0, x], dim = 'model')
        
        # note model concatenated
        models += [x.model]
        
        # close Dataset object
        x.close()
        del(x)
        
    # update attribute information to concatenated Dataset
    x0.attrs['model_index'] = models # add names of model concatenated in order of concatenation;
                                     # index of model dimension will match names in list
    del(x0.attrs['model'], x0.attrs['modelling_group'], x0.attrs['realisation'])
        
    # save Dataset object containing the concatenated model output for specfic group
    save_pw = '/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/processed/' + \
              f'{x0.experiment}_{x0.data_type[0: 4]}_all_models.nc'
    x0.to_netcdf(save_pw, 'w')
    print(f'File saved: {save_pw.split("/")[-1]}')
    del(save_pw)
    
    # create a dictionary object storing attribute information for new objects
    attrs_info = {'models': models, 'models_used': len(models),
                  'data_type': x0.data_type, 'experiment': x0.experiment,
                  'year_sampled': x0.years_sampled, 'period': x0.period, 'resolution': x0.resolution}

    # compute the ensemble spatial annual heat exposure variables and add attribute information
    if 'anomaly' in x0.data_type:
        ens_mean = x0.mean_exposure_anom.mean(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_intra = x0.std_exposure_anom.mean(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_std = x0.mean_exposure_anom.std(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_qua = x0.mean_exposure_anom.quantile([0.05, 0.25, 0.75, 0.95], dim = 'model',
                                                 interpolation = 'linear').assign_attrs(attrs_info)
        ens_medi = x0.mean_exposure_anom.median(dim = 'model', skipna = True).assign_attrs(attrs_info)
    
    elif 'absolute' in x0.data_type:
        ens_mean = x0.mean_exposure.mean(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_intra = x0.std_exposure.mean(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_std = x0.mean_exposure.std(dim = 'model', skipna = True).assign_attrs(attrs_info)
        ens_qua = x0.mean_exposure.quantile([0.05, 0.25, 0.75, 0.95], dim = 'model',
                                          interpolation = 'linear').assign_attrs(attrs_info)
        ens_medi = x0.mean_exposure.median(dim = 'model', skipna = True).assign_attrs(attrs_info)
    
    else:
        raise ValueError('check data type attribute of Datasets being averaged.')
        
    # add DataArray objects to ensemble Dataset object
    ens_ds = ens_ds.assign({f'{x0.experiment}_mean_exposure_{x0.data_type[0: 4]}': ens_mean})
    ens_ds = ens_ds.assign({f'{x0.experiment}_intrvar_exposure_{x0.data_type[0: 4]}': ens_intra})
    ens_ds = ens_ds.assign({f'{x0.experiment}_std_exposure_{x0.data_type[0: 4]}': ens_std})
    ens_ds = ens_ds.assign({f'{x0.experiment}_quantiles_exposure_{x0.data_type[0: 4]}': ens_qua})
    ens_ds = ens_ds.assign({f'{x0.experiment}_medi_exposure_{x0.data_type[0: 4]}': ens_medi})
    
# update attribute informaton to Dataset object
ens_ds = ens_ds.assign_attrs(attrs_info)
del(ens_ds.attrs['data_type'], ens_ds.attrs['period'], ens_ds.attrs['experiment'],
    ens_ds.attrs['year_sampled'])
ens_ds.attrs['description'] = '"mean_exposure" = spatial ensemble mean annual heat exposure; ' +\
                              '"intravar_exposure" = spatial ensemble mean model inter-annual variabiltiy in '+\
                              'annual heat exposure; "std_exposure" = spatial ensemble standard '+\
                              'deviation in annual heat exposure; "quantiles_exposure" = 5th, 25th, 75th '+\
                              'and 95th percentiles of ensemble annual heat exposure; "medi_exposure" = '+\
                              'spatial ensemble median annual heat exposure.'

# save Dataset object
save_pw = '/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/processed/ensemble_heat_exposure.nc'
ens_ds.to_netcdf(save_pw, 'w')
print(f'File saved: {save_pw.split("/")[-1]}')

# close DataArray/set objects
x0.close(), ens_ds.close(), ens_mean.close(), ens_intra.close(), ens_std.close(),
ens_qua.close(), ens_medi.close()

File saved: current_anom_all_models.nc
File saved: ssp126_anom_all_models.nc
File saved: ssp245_anom_all_models.nc
File saved: ssp370_anom_all_models.nc
File saved: ssp585_anom_all_models.nc
File saved: pre_ind_abso_all_models.nc
File saved: current_abso_all_models.nc
File saved: ssp126_abso_all_models.nc
File saved: ssp245_abso_all_models.nc
File saved: ssp370_abso_all_models.nc
File saved: ssp585_abso_all_models.nc
File saved: ensemble_heat_exposure.nc


(None, None)

### Absolute Heat Exposure and Heat Exposure Anomaly: Regional Aggregated Results

NOT WEIGHTED MEANS BUT INFACT need to sum exposure using region masks, but summation can be done for all varaibles, i.e. mean, median, etc. no weighting requried

In [20]:
def weighted_mean(arr, weights, dim):
    '''
    Summary:
    --------
    Computes a weighted mean of an array along a dimension/dimensions of a DataArray object.
    
    Parameters:
    -----------
    arr: xarray DataArray object
         array containing values used to compute a weighted mean
         
    weights: xarray DataArray object
             array containing weights to be applied to the values within arr object
             
    dim: str or sequence of str
         dimension/s over which to compute the weighted mean
         
    Returns:
    --------
    weighted_mean: xarray DataArray oject
                   array containing weighted mean with the dimension/s mean calculated over removed.
                   
    '''
    
    # sum up the weighted sum of the values within the region specified by dim
    # recall matrix multiplication; cols of first equal rows of second; (10x2) * (2x6) = (10x6)
    # so weights array does not need to be same shape as data array
    weighted_sum = (arr * weights).sum(dim = dim, skipna = True)
    
    # sum up the values of the weights within the region specified by dim
    # define an array where weights of cells with valid and invalid values are preserved and NA respectively
    masked_weights = weights.where(arr.notnull()) 
    
    # sum up the weights of the valid cells
    sum_of_weights = masked_weights.sum(dim = dim, skipna = True)
    
    # as cannot divide by zero, set weights equal to zero as NA
    # the values of these cells will be removed from the mean in the weighted sum part (multiplied by 0)
    valid_weights = sum_of_weights != 0
    sum_of_weights = sum_of_weights.where(valid_weights)
    
    # compute weighted mean along the specified dimension/s
    weighted_mean = weighted_sum / sum_of_weights
    
    return weighted_mean

In [21]:
def region_average(arr, regions, land_only = True):
    '''
    Summary:
    --------
    Computes weighted mean of an array for various regions, as well for the globe, the ocean, 
    and the land with and without Antartica.
    
    Parameters:
    -----------
    arr: xarray DataArray object
         array encompassing the regions where the weighted means are computed
    
    regions: regionmask.Regions object
             regions to compute the weighted means over
        
    land_only : bool
                whether to mask out ocean points before calculating regional means
                default is True
        
    Returns:
    --------
    reg_ave : xarray DataArray object
              New DataArray with weighted mean over the whole globe, the ocean, the land, the 
              land without Antarctica, and all regions.
              Dimensions (n_regions + 4) x (additional dimensions no averaged over)
    
    '''
    # check that regions specified are an instance of regionmask.Regions;
    # essnetially checking all are an 'regionmask.Regions' object type
    if not isinstance(regions, regionmask.Regions):
        raise ValueError('specified regions must be a regionmask.Regions instance')
        
    # define names of regions to be used to index the various weighted means to be computed
    abbrevs = ['global', 'ocean', 'land', 'land_wo_antarctica']
    abbrevs = abbrevs + regions.abbrevs
    
    # define the IPCC numbers for each regions to index the various weighted means to be computed
    numbers = np.array(regions.numbers)
    
    # compute the latitude weights using the cosine approximation
    weight = np.cos(np.deg2rad(arr.lat))
    
    # define a land mask where land and sea cells are True and False respectively
    landmask = regionmask.defined_regions.natural_earth.land_110.mask(arr)
    landmask = landmask == 0
    
    # for land only, combine latitude weighting with landmask
    # result being that only cells over land have non-zero weights following latitude cosine approximation
    if land_only:
        wgt = weight * landmask
    
    # otherwise, combine latitude weighting with same shape as input array
    # result being that all cells, both land or ocean, will have a weight corresponding to its latitude
    # this weight will be used for regions containing both ocean and land cells
    else:
        wgt = xr.full_like(landmask, 1) * weight
    
    # define a region mask; cells given number for a given region (Europe = 1, Aus = 2 etc.)
    # cells that do not fall into a region, denoted as NaN
    mask = regions.mask(arr)
    
    # define a list to accumulate averages/weighted means
    ave = list()
    
    # compute global mean
    # weighting is simple cosine latitude weighting; no mask as averging entire surface/all cells
    a = weighted_mean(arr, dim = ('lat', 'lon'), weights = weight)
    ave.append(a)
    
    # compute global ocean mean
    # weighting is a cosine latitude weighting of only ocean cells; land cells all weighted as 0
    weights = (weight * (1.0 - landmask))
    a = weighted_mean(arr, dim = ('lat', 'lon'), weights = weights)
    ave.append(a)
    
    # compute global land mean
    # weighting is a cosine latitude weighting of only land cells; ocean cells weighted as 0
    weights = (weight * landmask)
    a = weighted_mean(arr, dim = ('lat', 'lon'), weights = weights)
    ave.append(a)
    
    # compute global land mean without Antarctica
    # weighting is a cosine latitude weighting of only land cells; ocean cells weighted as 0
    arr_selected = arr.sel(lat = slice(-60, None)) # remove Antarctica by removing low latitudes
    weights = (weight * landmask)
    a = weighted_mean(arr_selected, dim = ('lat', 'lon'), weights = weights)
    ave.append(a)
    
    #### Regional Weighted Means ####
    # Computing the specified regional averages is quicker using Groupby objects
    # Groupby objects use multi indexing of coordinates to state the raster cells of a given group
    # Multi indexing essentially concatenates the coordinates
    # (i.e. a cell with lat = -5 and lon = 25 denoted as lat_lon = -5, 25)
    # Using these "stacked" coordinates reduces the dimensions/shape of an array
    # (i.e. stacking lat and lon will change the 2D representation of these to 1D)
    
    # compute the region weighted means
    g = arr.groupby(mask) # group array into the different regions
    
    # create a new dimension of 'stacked coordinates'; moves from raster/grid format to 1D object
    # (unstacked dimensions -> lat = 10, lon = 12, value = 50 stacked dimension -> 10_12, value = 50)
    wgt_stacked = wgt.stack(stacked_lat_lon = ('lat', 'lon'))
    
    # apply stacked lat_lon weights to stacked Groupby object
    a = g.apply(weighted_mean, dim = ('stacked_lat_lon'), weights = wgt_stacked)

    ave.append(a.drop('region')) # drop the region information as want to use as dimension to merge averages
    
    # merge the list of weighted means DataArray objects into a single DataArray object
    arr = xr.concat(ave, dim = 'region')
    
    # shift region coordinates such that the numbers correspond to the regions
    # accounting for the 4 non-regional weighted means also computed
    numbers = np.arange(numbers.min() - 4, numbers.max() + 1)
    
    # add the abbreviations of the regions and update the numbers
    arr = arr.assign_coords(**{'abbrev': ('region', abbrevs), 'number': ('region', numbers)})
    
    # create a multi index
    arr = arr.set_index(region = ('abbrev', 'number'))
    
    return arr

In [24]:
# load in spatial mulit-model ensemble annual heat exposure variables; has both absolute and anomaly
f_pw = '/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/processed/ensemble_heat_exposure.nc'
data = xr.open_dataset(f_pw)

# define AR6 land regions to calculate weighted means for; must be regionmask objects
regions = regionmask.defined_regions.ar6.land

# create DataFrame object to store weighted mean
wm_df = pd.DataFrame()

# define column headers
col_names = ['global', 'ocean', 'land', 'land_wo_antarctica']
col_names = col_names + regions.abbrevs[: -2]

# add column headers to DataDrame object
wm_df = wm_df.reindex(columns = col_names)

# create a list of the various DataArray objects
data_vars = data.data_vars.values() # .data_vars gives a dictionary object; .values() lists the DataArrays

# loop over each DataArray object
for data_arr in data_vars:
    
    # compute the regional weighted means; use only land cells for mean with regions with both ocean and land
    w_means = region_average(data_arr, regions, land_only = True)
    
    # must loop over each percentile with quantile DataArray
    if 'quantiles' in data_arr.name:
        for i in range(0, 4):
            
            # add a row to DataFrame object for each percentile
            row_name = f'{data_arr.name}_pct_{str(data_arr["quantile"][i].values)[2:]}'
            wm_df = wm_df.append(pd.Series(name = row_name, dtype = 'float64'))
            
            # populate the DataFrame object for each percentile
            for col_name in col_names:
                wm_df[col_name][row_name] = w_means[i, :].sel(abbrev = col_name)
    
    
    elif 'quantiles' not in data_arr.name:
        
        # add a row to DataFrame object
        row_name = data_arr.name
        wm_df = wm_df.append(pd.Series(name = row_name, dtype = 'float64'))
      
        # populate the DataFrame object
        for col_name in col_names:
            wm_df[col_name][row_name] = w_means.sel(abbrev = col_name)
            
# save DataFrame object using pickle; deconstructs and reconstucts data to save space
save_pw = '/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/processed/ensemble_heat_exposure_variables_AR6_oldway.pickle'
with open(save_pw, 'wb') as f:
    pickle.dump(wm_df, f) 
print(f'File saved: {save_pw.split("/")[-1]}')

# close Dataset object    
data.close()
del(data)

File saved: ensemble_heat_exposure_variables_AR6.pickle


In [35]:
print(regions)

<regionmask.Regions>
Name:     AR6 reference regions (land only)
Source:   Iturbide et al., 2020 (Earth Syst. Sci. Data)

Regions:
  0  GIC  Greenland/Iceland
  1  NWN  N.W.North-America
  2  NEN  N.E.North-America
  3  WNA    W.North-America
  4  CNA    C.North-America
..   ...                ...
 41  EAU        E.Australia
 42  SAU        S.Australia
 43   NZ        New-Zealand
 44  EAN       E.Antarctica
 45  WAN       W.Antarctica

[46 regions]


In [25]:
# load in spatial mulit-model ensemble annual heat exposure variables; has both absolute and anomaly
ens_pw = '/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/processed/ensemble_heat_exposure.nc'
ens_ds = xr.open_dataset(ens_pw)

# define an AR6 land region mask for common 1x1 degree resolution data has been regridded to 
lon, lat = np.arange(-179.5, 180), np.arange(-89.5, 90)
mask = regionmask.defined_regions.ar6.land.mask(lon, lat) # grid cells encoded with AR6 region number

# define AR6 land regions to sum over; must be regionmask objects
regions = regionmask.defined_regions.ar6.land

# create DataFrame object to store regional aggregated variables
reg_df = pd.DataFrame()

# define column headers
col_names = ['land', 'land_wo_antarctica']
col_names = col_names + regions.abbrevs[: -2]

# add column headers to DataDrame object
reg_df = reg_df.reindex(columns = col_names)

# create a list of the various DataArray objects
data_vars = ens_ds.data_vars.values()

# loop over each DataArray object; the ensemble heat exposure variables
for data_arr in data_vars:
    
    if 'quantiles' not in data_arr.name:
          
        # add a row to DataFrame object for given heat exposure variable
        row_name = data_arr.name
        reg_df = reg_df.append(pd.Series(name = row_name, dtype = 'float64'))
    
        # loop over each AR6 land region; last two are Antartica land regions so dismiss 
        for region_id, region_abbrev in zip(regions.numbers[: -2], regions.abbrevs[: -2]):

            # apply region mask; sets all cells outside regions to nan
            region_data = data_arr.where(mask == region_id)
            
            # sum up all values within region; populate DataFrame object
            col_name = region_abbrev 
            reg_df[col_name][row_name] = region_data.sum(skipna = True)
            
        # compute global land summation
        col_name = 'land'
        global_land_data = data_arr.where(mask.fillna(-999) != -999) # sets all non-land cells to nan
        reg_df[col_name][row_name] = global_land_data.sum(skipna = True)

        # compute global land summation without Antarctica
        col_name = 'land_wo_antarctica'
        global_land_wo_antarctica_data = global_land_data.sel(lat = slice(-60, 90)) # cut off Antarctica
        reg_df[col_name][row_name] = global_land_wo_antarctica_data.sum(skipna = True)
    
    if 'quantiles' in data_arr.name:
        for i in range(0, 4):
            
            # add a row to DataFrame object for each percentile
            row_name = f'{data_arr.name}_pct_{str(data_arr["quantile"][i].values)[2: ]}'
            reg_df = reg_df.append(pd.Series(name = row_name, dtype = 'float64'))
            
            # loop over each AR6 land region; last two are Antartica land regions so dissmiss 
            for region_id, region_abbrev in zip(regions.numbers[: -2], regions.abbrevs[: -2]):

                # apply region mask; sets all cells outside regions to nan
                region_data = data_arr.where(mask == region_id)
            
                # populate the DataFrame object for each percentile
                col_name = region_abbrev
                reg_df[col_name][row_name] = region_data[i, :, :].sum(skipna = True)
            
            # compute global land summation
            col_name = 'land'
            global_land_data = data_arr.where(mask.fillna(-999) != -999) # sets all non-land cells to nan
            reg_df[col_name][row_name] = global_land_data[i, :, :].sum(skipna = True)

            # compute global land summation without Antarctica
            col_name = 'land_wo_antarctica'
            global_land_wo_antarctica_data = global_land_data.sel(lat = slice(-60, 90)) # cut off Antarctica
            reg_df[col_name][row_name] = global_land_wo_antarctica_data[i, :, :].sum(skipna = True)
        
    # close open Dataset and/or DataArray objects
    data_arr.close(), global_land_data.close(), global_land_wo_antarctica_data.close()
    
# save DataFrame object using pickle; deconstructs and reconstucts data to save space
save_pw = '/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/processed/ensemble_heat_exposure_variables_AR6.pickle'
with open(save_pw, 'wb') as f:
    pickle.dump(reg_df, f) 
print(f'File saved: {save_pw.split("/")[-1]}')

# close open Dataset and/or DataArray objects
ens_ds.close()

File saved: ensemble_heat_exposure_variables_AR6.pickle


In [22]:
reg_df

,land,land_wo_antarctica,GIC,NWN,NEN,WNA,CNA,ENA,NCA,SCA,...,TIB,EAS,ARP,SAS,SEA,NAU,CAU,EAU,SAU,NZ
current_mean_exposure_anom,6.203562e+11,6.203562e+11,0.000000,2.893046e+06,3.886343e+06,5.272996e+08,3.868227e+09,9.689725e+09,4.424168e+09,1.146561e+10,...,7.155836e+09,8.091756e+10,7.329382e+09,1.761115e+11,1.271308e+11,3.251090e+08,2.760641e+07,7.374137e+08,1.768518e+08,3.399262e+07
current_intrvar_exposure_anom,5.021808e+10,5.021808e+10,0.000000,3.745264e+06,2.930543e+06,1.814887e+08,5.461199e+08,1.742707e+09,6.154517e+08,7.272097e+08,...,7.143840e+08,1.204178e+10,4.719638e+08,1.071650e+10,2.359639e+09,7.596499e+06,2.570748e+06,9.457959e+07,5.503054e+07,1.969706e+07
current_std_exposure_anom,1.055276e+11,1.055276e+11,0.000000,3.237618e+06,3.340603e+06,4.680341e+08,7.964110e+08,2.588635e+09,1.990840e+09,2.474793e+09,...,2.720032e+09,1.771556e+10,1.482787e+09,2.436288e+10,5.255790e+09,1.873827e+07,4.406441e+06,2.233251e+08,7.450269e+07,2.443966e+07
current_quantiles_exposure_anom_pct_05,4.656257e+11,4.656257e+11,0.000000,4.198787e+05,8.064667e+05,7.134232e+07,2.717741e+09,5.654318e+09,2.062669e+09,7.806641e+09,...,3.196120e+09,5.231748e+10,5.144275e+09,1.400485e+11,1.188634e+11,2.935347e+08,2.078838e+07,4.559140e+08,8.543892e+07,9.684844e+06
current_quantiles_exposure_anom_pct_25,5.557763e+11,5.557763e+11,0.000000,8.527248e+05,1.588517e+06,1.651756e+08,3.428018e+09,8.201047e+09,3.210919e+09,9.813936e+09,...,5.676063e+09,6.903111e+10,6.373543e+09,1.615969e+11,1.251766e+11,3.130036e+08,2.521062e+07,5.775123e+08,1.203063e+08,1.818908e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ssp585_quantiles_exposure_abso_pct_05,1.353732e+12,1.353732e+12,0.000000,6.670039e+07,4.333941e+07,2.612346e+09,1.472640e+10,4.214393e+10,1.003258e+10,1.661815e+10,...,1.273219e+10,1.185683e+11,3.112094e+10,3.385522e+11,1.811418e+11,8.297148e+08,7.753046e+07,3.877686e+09,1.187020e+09,2.828441e+08
ssp585_quantiles_exposure_abso_pct_25,1.569284e+12,1.569284e+12,0.000000,1.276730e+08,6.300737e+07,4.563693e+09,1.664960e+10,5.094365e+10,1.273482e+10,1.914631e+10,...,1.907172e+10,1.375144e+11,3.596169e+10,3.867565e+11,1.849143e+11,8.720093e+08,9.415297e+07,4.740304e+09,1.586938e+09,3.991363e+08
ssp585_quantiles_exposure_abso_pct_75,1.834492e+12,1.834492e+12,23.364385,3.955778e+08,1.311513e+08,8.968634e+09,2.046163e+10,6.462172e+10,1.884533e+10,2.240818e+10,...,2.507614e+10,1.601556e+11,4.131529e+10,4.401546e+11,1.877533e+11,9.162323e+08,1.061213e+08,6.118084e+09,2.497187e+09,7.310310e+08
ssp585_quantiles_exposure_abso_pct_95,1.965763e+12,1.965763e+12,40724.976205,5.524209e+08,1.593418e+08,1.180706e+10,2.179252e+10,7.276506e+10,2.413369e+10,2.406395e+10,...,2.762724e+10,1.732185e+11,4.427965e+10,4.625878e+11,1.887308e+11,9.299299e+08,1.118755e+08,6.961045e+09,3.334010e+09,9.358707e+08


### Table View of Regional Heat Exposure

In [15]:
# load data weighted regional means data
exp_pw = '/home/ucfagtj/DATA/Dissertation/Data/heat_exposure/processed/ensemble_heat_exposure_variables_AR6.pickle'
unpickle = open(exp_pw, 'rb')
exp_df = pickle.load(unpickle)

# create a DataFrame object with Regions and periods column
col_names = ['region', 'pre_ind', 'current', 'ssp126', 'ssp245', 'ssp370', 'ssp585', 
             'current_anom', 'ssp126_anom', 'ssp245_anom', 'ssp370_anom', 'ssp585_anom',
             'current_pct', 'ssp126_pct', 'ssp245_pct', 'ssp370_pct', 'ssp585_pct']
df = pd.DataFrame()
df = df.reindex(columns = col_names)

# define region names; excluding the two Antartica land regions
region_names = ['land', 'land_wo_antarctica'] + regionmask.defined_regions.ar6.land.abbrevs[: -2]

# extract absolute and percentage increase for each region
for i, region in enumerate(region_names):
    
    # restrict exposure DataFrame object to given region
    reg_data = exp_df[f'{region}']
    
    # extract absolute values
    pre_ind = reg_data['pre_ind_mean_exposure_abso']
    current = reg_data['current_mean_exposure_abso']
    ssp126 = reg_data['ssp126_mean_exposure_abso']
    ssp245 = reg_data['ssp245_mean_exposure_abso']
    ssp370 = reg_data['ssp370_mean_exposure_abso']
    ssp585 = reg_data['ssp585_mean_exposure_abso']
    
    # extract pre-industrial anomaly values
    current_anom = reg_data['current_mean_exposure_anom']
    ssp126_anom = reg_data['ssp126_mean_exposure_anom']
    ssp245_anom = reg_data['ssp245_mean_exposure_anom']
    ssp370_anom = reg_data['ssp370_mean_exposure_anom']
    ssp585_anom = reg_data['ssp585_mean_exposure_anom']
    
    # calculate percentage change of anomaly relative to pre-industrial value
    if pre_ind == 0:
        current_pct, ssp126_pct, ssp245_pct, ssp370_pct, ssp585_pct = '-', '-', '-', '-', '-'
        
    else:
        current_pct = round((current_anom / pre_ind) * 100, 1)
        ssp126_pct = round((ssp126_anom / pre_ind) * 100, 1) 
        ssp245_pct = round((ssp245_anom / pre_ind) * 100, 1)
        ssp370_pct = round((ssp370_anom / pre_ind) * 100, 1)
        ssp585_pct = round((ssp585_anom / pre_ind) * 100, 1)
        
    # from Dictionary object holding a given region's data; quote values as multiples of a million
    latex_bit = '\multirow{2}{*}{'
    data = {'region': latex_bit + f'{region}' + '}',
            'pre_ind': latex_bit + f'{round(pre_ind / 1e8, 3)}' + '}',
            'current': latex_bit + f'{round(current / 1e8, 3)}' + '}',
            'current_anom': round(current_anom / 1e8, 3),
            'ssp126': latex_bit + f'{round(ssp126 / 1e8, 3)}' + '}', 
            'ssp126_anom': round(ssp126_anom / 1e8, 3),
            'ssp245': latex_bit + f'{round(ssp245 / 1e8, 3)}' + '}',
            'ssp245_anom': round(ssp245_anom / 1e8, 3),
            'ssp370': latex_bit + f'{round(ssp370 / 1e8, 3)}' +'}',
            'ssp370_anom': round(ssp370_anom / 1e8, 3),
            'ssp585': latex_bit + f'{round(ssp585 / 1e8, 3)}' + '}',
            'ssp585_anom': round(ssp585_anom / 1e8, 3),
            'current_pct': f'({current_pct}\%)',
            'ssp126_pct': f'({ssp126_pct}\%)', 'ssp245_pct': f'({ssp245_pct}\%)',
            'ssp370_pct': f'({ssp370_pct}\%)', 'ssp585_pct': f'({ssp585_pct}\%)'}
    
    # add data as new entry to DataFrame object
    df = df.append(data, ignore_index = True, sort = False)
df

,region,pre_ind,current,ssp126,ssp245,ssp370,ssp585,current_anom,ssp126_anom,ssp245_anom,ssp370_anom,ssp585_anom,current_pct,ssp126_pct,ssp245_pct,ssp370_pct,ssp585_pct
0,\multirow{2}{*}{land},\multirow{2}{*}{1464.442},\multirow{2}{*}{7668.549},\multirow{2}{*}{13381.254},\multirow{2}{*}{18301.991},\multirow{2}{*}{26739.576},\multirow{2}{*}{16908.2},6203.562,11915.798,16836.049,25272.719,15442.167,(423.6\%),(813.7\%),(1149.7\%),(1725.8\%),(1054.5\%)
1,\multirow{2}{*}{land_wo_antarctica},\multirow{2}{*}{1464.442},\multirow{2}{*}{7668.549},\multirow{2}{*}{13381.254},\multirow{2}{*}{18301.991},\multirow{2}{*}{26739.576},\multirow{2}{*}{16908.2},6203.562,11915.798,16836.049,25272.719,15442.167,(423.6\%),(813.7\%),(1149.7\%),(1725.8\%),(1054.5\%)
2,\multirow{2}{*}{GIC},\multirow{2}{*}{0.0},\multirow{2}{*}{0.0},\multirow{2}{*}{0.0},\multirow{2}{*}{0.0},\multirow{2}{*}{0.0},\multirow{2}{*}{0.0},0.000,0.000,0.000,0.000,0.000,(-\%),(-\%),(-\%),(-\%),(-\%)
3,\multirow{2}{*}{NWN},\multirow{2}{*}{0.001},\multirow{2}{*}{0.03},\multirow{2}{*}{0.225},\multirow{2}{*}{0.503},\multirow{2}{*}{0.649},\multirow{2}{*}{2.728},0.029,0.224,0.502,0.648,2.727,(2630.1\%),(20349.1\%),(45606.4\%),(58920.1\%),(247881.8\%)
4,\multirow{2}{*}{NEN},\multirow{2}{*}{0.003},\multirow{2}{*}{0.042},\multirow{2}{*}{0.19},\multirow{2}{*}{0.304},\multirow{2}{*}{0.254},\multirow{2}{*}{0.984},0.039,0.186,0.301,0.251,0.981,(1210.7\%),(5805.1\%),(9375.7\%),(7805.8\%),(30567.8\%)
5,\multirow{2}{*}{WNA},\multirow{2}{*}{0.116},\multirow{2}{*}{5.389},\multirow{2}{*}{19.419},\multirow{2}{*}{26.64},\multirow{2}{*}{25.274},\multirow{2}{*}{69.886},5.273,19.304,26.525,25.159,69.771,(4562.4\%),(16702.0\%),(22950.1\%),(21768.2\%),(60367.8\%)
6,\multirow{2}{*}{CNA},\multirow{2}{*}{7.107},\multirow{2}{*}{45.789},\multirow{2}{*}{97.392},\multirow{2}{*}{105.852},\multirow{2}{*}{74.694},\multirow{2}{*}{184.068},38.682,90.285,98.745,67.587,176.961,(544.3\%),(1270.4\%),(1389.4\%),(951.0\%),(2489.9\%)
7,\multirow{2}{*}{ENA},\multirow{2}{*}{16.142},\multirow{2}{*}{113.039},\multirow{2}{*}{274.57},\multirow{2}{*}{311.63},\multirow{2}{*}{233.278},\multirow{2}{*}{572.567},96.897,258.428,295.487,217.135,556.424,(600.3\%),(1601.0\%),(1830.5\%),(1345.1\%),(3447.0\%)
8,\multirow{2}{*}{NCA},\multirow{2}{*}{3.599},\multirow{2}{*}{47.842},\multirow{2}{*}{88.654},\multirow{2}{*}{125.772},\multirow{2}{*}{190.198},\multirow{2}{*}{163.199},44.242,85.054,122.171,186.595,159.598,(1229.1\%),(2363.0\%),(3394.2\%),(5184.1\%),(4434.0\%)
9,\multirow{2}{*}{SCA},\multirow{2}{*}{10.532},\multirow{2}{*}{125.203},\multirow{2}{*}{191.892},\multirow{2}{*}{282.493},\multirow{2}{*}{494.673},\multirow{2}{*}{207.388},114.656,181.334,271.924,484.076,196.826,(1088.6\%),(1721.7\%),(2581.8\%),(4596.1\%),(1868.8\%)


In [20]:
df.iloc[[17, 30, 45]]

,region,pre_ind,current,ssp126,ssp245,ssp370,ssp585,current_anom,ssp126_anom,ssp245_anom,ssp370_anom,ssp585_anom,current_pct,ssp126_pct,ssp245_pct,ssp370_pct,ssp585_pct
17,\multirow{2}{*}{SSA},\multirow{2}{*}{0.001},\multirow{2}{*}{0.032},\multirow{2}{*}{0.03},\multirow{2}{*}{0.058},\multirow{2}{*}{0.127},\multirow{2}{*}{0.086},0.031,0.029,0.057,0.126,0.085,(4088.1\%),(3799.1\%),(7463.6\%),(16408.2\%),(11142.8\%)
30,\multirow{2}{*}{RAR},\multirow{2}{*}{0.0},\multirow{2}{*}{0.001},\multirow{2}{*}{0.004},\multirow{2}{*}{0.011},\multirow{2}{*}{0.033},\multirow{2}{*}{0.041},0.001,0.004,0.010,0.033,0.041,(375.7\%),(1656.0\%),(4656.3\%),(14590.1\%),(18092.2\%)
45,\multirow{2}{*}{NZ},\multirow{2}{*}{0.026},\multirow{2}{*}{0.366},\multirow{2}{*}{1.421},\multirow{2}{*}{2.336},\multirow{2}{*}{2.278},\multirow{2}{*}{5.786},0.340,1.395,2.310,2.252,5.759,(1302.4\%),(5344.9\%),(8849.1\%),(8627.1\%),(22066.1\%)
